# Step 2. Preprocessing CSV file. (df가 원본csv의 16행부터 시작하도록)

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
data_dir = Path("./csv")
csv_files = sorted(data_dir.glob("*.csv"))

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {data_dir.resolve()}")

# 각 CSV 파일을 개별 DataFrame으로 읽어 별도 변수와 컬렉션에 저장
dfs_map = {}   # 파일명(stem) -> DataFrame
dfs_list = []  # index 순서의 리스트

for i, fp in enumerate(csv_files):
    df_i = pd.read_csv(fp)
    dfs_list.append(df_i)
    dfs_map[fp.stem] = df_i
    # 개별 변수로도 접근 가능하게 전역 네임스페이스에 저장 (예: df_0, df_1, ...)
    globals()[f"df_{i}"] = df_i

In [3]:
dfs_map.keys()  # 파일명(stem) 리스트

dict_keys(['(2015) 10월말+주민등록인구+및+외국인+현황 (1)', '(2015) 11월말+주민등록인구+및+외국인+현황 (1)', '(2015) 12월말+주민등록인구+및+외국인+현황 (1)', '(2015) 1월말+주민등록인구+및+외국인+현황', '(2015) 2월말+주민등록인구+및+외국인+현황', '(2015) 3월말+주민등록인구+및+외국인+현황', '(2015) 4월말+주민등록인구+및+외국인+현황 (1)', '(2015) 5월말+주민등록인구+및+외국인+현황 (1)', '(2015) 6월말+주민등록인구+및+외국인+현황 (1)', '(2015) 7월말+주민등록인구+및+외국인+현황 (1)', '(2015) 8월말+주민등록인구+및+외국인+현황 (1)', '(2015) 9월말+주민등록인구+및+외국인+현황 (1)', '(2016) 10월말+주민등록인구+및+외국인+현황', '(2016) 11월말+주민등록인구+및+외국인+현황', '(2016) 12월말+주민등록인구+및+외국인+현황', '(2016) 1월말+주민등록인구+및+외국인+현황', '(2016) 2월말+주민등록인구+및+외국인+현황', '(2016) 3월말+주민등록인구+및+외국인+현황', '(2016) 4월말+주민등록인구+및+외국인+현황', '(2016) 5월말+주민등록인구+및+외국인+현황', '(2016) 6월말+주민등록인구+및+외국인+현황', '(2016) 7월말+주민등록인구+및+외국인+현황', '(2016) 8월말+주민등록인구+및+외국인+현황', '(2016) 9월말+주민등록인구+및+외국인+현황', '(2022. 10월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 11월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 12월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 1월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 2월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 3월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 4월말) 창

In [4]:
def process_csv_with_header_at_row(df):
    """
    CSV 파일에서 특정 행을 컬럼명으로 하고, 그 다음 행부터를 데이터로 하는 데이터프레임 생성
    
    Args:
        df: 원본 데이터프레임
        header_row_idx: 헤더로 사용할 행의 인덱스, 해당 행은 다음과 같은 과정을 거쳐 추출됨.
        1. n행 2열에 "소계" 라는 단어가 있는 행을 찾음.
        2. 해당 행렬좌표가 (n, m) 라면, n행을 헤더로 사용하고, n+1행부터 끝까지를 데이터로 사용함.
    
    Returns:
        새로운 데이터프레임
    """
    # 헤더 행 추출
    header_row = df[df.iloc[:, 1] == "소계"].iloc[0]
    header_row_idx = header_row.name  # 행 인덱스
    # 새로운 데이터프레임 생성
    new_df = df.iloc[header_row_idx + 1:].copy()
    new_df.columns = header_row  # 컬럼명 설정
    new_df.reset_index(drop=True, inplace=True)  # 인덱스 재설정
    # 1,2열만 남기고 나머지 열 삭제
    new_df = new_df.iloc[:, :2]
    # 1열 컬럼명을 "행정동"으로 변경
    new_df.rename(columns={new_df.columns[0]: "행정동"}, inplace=True)
    return new_df

In [ ]:
# 첫 번째 파일로 테스트
df_processed = process_csv_with_header_at_row(dfs_list[0])
print(f"원본 데이터프레임 크기: {dfs_list[0].shape}")
print(f"처리된 데이터프레임 크기: {df_processed.shape}")
print("\n컬럼명:")
print(df_processed.columns.tolist())

In [ ]:
# 처리된 데이터프레임 확인
print("처리된 데이터프레임 상위 5개 행:")
df_processed.head()

## 전처리 완료한 dataframe 생성하기
processed_dfs_list 변수에 저장 완료.

In [5]:
# 모든 CSV 파일에 대해 동일한 처리 적용
processed_dfs_map = {}   # 파일명(stem) -> 처리된 DataFrame
processed_dfs_list = []  # index 순서의 처리된 DataFrame 리스트

for i, (fp, original_df) in enumerate(zip(csv_files, dfs_list)):
    processed_df = process_csv_with_header_at_row(original_df)
    processed_dfs_list.append(processed_df)
    processed_dfs_map[fp.stem] = processed_df
    # 개별 변수로도 접근 가능하게 전역 네임스페이스에 저장 (예: processed_df_0, processed_df_1, ...)
    globals()[f"processed_df_{i}"] = processed_df

print(f"총 {len(processed_dfs_list)}개의 CSV 파일이 처리되었습니다.")
print(f"처리된 파일들: {list(processed_dfs_map.keys())}")

총 128개의 CSV 파일이 처리되었습니다.
처리된 파일들: ['(2015) 10월말+주민등록인구+및+외국인+현황 (1)', '(2015) 11월말+주민등록인구+및+외국인+현황 (1)', '(2015) 12월말+주민등록인구+및+외국인+현황 (1)', '(2015) 1월말+주민등록인구+및+외국인+현황', '(2015) 2월말+주민등록인구+및+외국인+현황', '(2015) 3월말+주민등록인구+및+외국인+현황', '(2015) 4월말+주민등록인구+및+외국인+현황 (1)', '(2015) 5월말+주민등록인구+및+외국인+현황 (1)', '(2015) 6월말+주민등록인구+및+외국인+현황 (1)', '(2015) 7월말+주민등록인구+및+외국인+현황 (1)', '(2015) 8월말+주민등록인구+및+외국인+현황 (1)', '(2015) 9월말+주민등록인구+및+외국인+현황 (1)', '(2016) 10월말+주민등록인구+및+외국인+현황', '(2016) 11월말+주민등록인구+및+외국인+현황', '(2016) 12월말+주민등록인구+및+외국인+현황', '(2016) 1월말+주민등록인구+및+외국인+현황', '(2016) 2월말+주민등록인구+및+외국인+현황', '(2016) 3월말+주민등록인구+및+외국인+현황', '(2016) 4월말+주민등록인구+및+외국인+현황', '(2016) 5월말+주민등록인구+및+외국인+현황', '(2016) 6월말+주민등록인구+및+외국인+현황', '(2016) 7월말+주민등록인구+및+외국인+현황', '(2016) 8월말+주민등록인구+및+외국인+현황', '(2016) 9월말+주민등록인구+및+외국인+현황', '(2022. 10월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 11월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 12월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 1월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 2월말) 창원시 주민등록인구 및 외국인 현황', '(2022. 3월말) 창원시 주민등록인구 및

In [6]:
# 처리 결과 확인
print("각 처리된 데이터프레임의 크기:")
for i, df in enumerate(processed_dfs_list):
    file_name = csv_files[i].stem
    print(f"{i}: {file_name} - {df.shape}")

# 각 처리된 데이터프레임의 컬럼 및 샘플 출력
for i, df in enumerate(processed_dfs_list):
    file_name = csv_files[i].stem
    print(f"\n[{i}] {file_name} - 컬럼:")
    print(df.columns.tolist())

    print(f"\n[{i}] {file_name} - 샘플 상위 3개 행:")
    print(df.head(3))

각 처리된 데이터프레임의 크기:
0: (2015) 10월말+주민등록인구+및+외국인+현황 (1) - (68, 2)
1: (2015) 11월말+주민등록인구+및+외국인+현황 (1) - (68, 2)
2: (2015) 12월말+주민등록인구+및+외국인+현황 (1) - (68, 2)
3: (2015) 1월말+주민등록인구+및+외국인+현황 - (68, 2)
4: (2015) 2월말+주민등록인구+및+외국인+현황 - (68, 2)
5: (2015) 3월말+주민등록인구+및+외국인+현황 - (68, 2)
6: (2015) 4월말+주민등록인구+및+외국인+현황 (1) - (68, 2)
7: (2015) 5월말+주민등록인구+및+외국인+현황 (1) - (68, 2)
8: (2015) 6월말+주민등록인구+및+외국인+현황 (1) - (68, 2)
9: (2015) 7월말+주민등록인구+및+외국인+현황 (1) - (68, 2)
10: (2015) 8월말+주민등록인구+및+외국인+현황 (1) - (68, 2)
11: (2015) 9월말+주민등록인구+및+외국인+현황 (1) - (68, 2)
12: (2016) 10월말+주민등록인구+및+외국인+현황 - (68, 2)
13: (2016) 11월말+주민등록인구+및+외국인+현황 - (68, 2)
14: (2016) 12월말+주민등록인구+및+외국인+현황 - (68, 2)
15: (2016) 1월말+주민등록인구+및+외국인+현황 - (68, 2)
16: (2016) 2월말+주민등록인구+및+외국인+현황 - (68, 2)
17: (2016) 3월말+주민등록인구+및+외국인+현황 - (68, 2)
18: (2016) 4월말+주민등록인구+및+외국인+현황 - (68, 2)
19: (2016) 5월말+주민등록인구+및+외국인+현황 - (68, 2)
20: (2016) 6월말+주민등록인구+및+외국인+현황 - (68, 2)
21: (2016) 7월말+주민등록인구+및+외국인+현황 - (68, 2)
22: (2016) 8월말+주민등록인구+및+외국인+현황 - (68, 2)
23: (20

In [7]:
# processed_dfs_list 변수에 전처리 완료한 데이터프레임들을 utf-8 csv로 저장
output_dir = Path("processed_data")
output_dir.mkdir(exist_ok=True)

# file_name을 다음과 같이 커스터마이징
"""
1. 원본 파일명에서 나타나는 숫자값을 배열로 추출.
2. 추출하면 아마도 20XXXX형태의 숫자 혹은 XXXX형태의 숫자가 나올것임.
3. 이때 XXXX형태의 숫자가 나오면 앞에 20을 붙여서 20XXXX형태로 만듦.
4. 그리고 나서, 파일명을 "resident_population_20XXXX" 형태로 저장함
"""
file_name_template = "resident_population_{}"



for i, df in enumerate(processed_dfs_list):
    # 원본 파일명에서 숫자 추출
    original_file_name = csv_files[i].stem
    
    # 파일명에 먼저나오는 순서대로 최대 6개의 숫자 추출.
    import re
    year = re.findall(r'\d+', original_file_name)
    year = "".join(year[:2])  # 최대 6자리 숫자 추출
    
    # 추출된 숫자가 3개이면 다음과 같이 처리. 예: 211 -> 202101
    # 추출된 숫자수가 4개이면 숫자면 20을 붙여서 20XXXX 형태로 변환. XXXXX같이 5자리면 다음의 예와 같이 처리. 예: 20151 -> 201501
    if len(year) == 3:
        year = "20" + year[:2] + "0" + year[2]
    elif len(year) == 4:
        year = "20" + year
    elif len(year) == 5:
        year = year[:4] + "0" + year[4]
    elif len(year) == 6:
        year = year
    else:
        raise ValueError(f"Unexpected year format in file name: {original_file_name}")
    
    # 최종 파일명 생성
    file_name = file_name_template.format(year)
    df.to_csv(output_dir / f"{file_name}_processed.csv", index=False, encoding="utf-8")